In [1]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import basicDeltaOperations as op
import pandas as pd
import copy

In [2]:
import math

def nCr(n,r):
    f = math.factorial
    return f(n) / f(r) / f(n-r)

def cleanUpElID(elID):
    '''
    quick function to pass correct string to the "delta value to concentration function", which distinguishes
    between 17O/18O, 33S/34S, etc. 
    '''
    if elID == 'S':
        subId = '33S'
    elif elID == 'O':
        subId = '17O'
        
    else:
        subId = elID
        
    return subId

def condenseStr(text):
    '''
    Condense the string depiction for easier calculation
    '''
    text = text.replace('(', '')
    text = text.replace(')', '')
    text = text.replace(',', '')
    text = text.replace(' ', '')
    
    return text
    
def fragMult(z, y):
    '''
    '''
    if z == 'x' or y == 'x':
        return 'x'
    else:
        return z*y
    
def intX(n):
    if n == 'x':
        return 'x'
    else:
        return int(n)
    
def expandFrag(frag, number):
    intermediate = []
    for i, v in enumerate(frag):
        intermediate += [v] * number[i]
    final = [intX(x) for x in intermediate]
    
    return final

def uEl(el, n):
    if n == 0:
        return ''
    if n == 'x':
        return ''
    if el == 'C':
        if n == 1:
            return '13C'
    if el == 'H':
        if n == 1:
            return 'D'
    if el == 'O':
        if n == 1:
            return '17O'
        if n == 2:
            return '18O'
    if el == 'N':
        if n == 1:
            return '15N'
    if el == 'S':
        if n == 1:
            return '33S'
        if n == 2:
            return '34S'
        if n == 4:
            return '36S'
        
subDict = {'C':{'0':'','1':'13C'},
           'N':{'0':'','1':'15N'},
           'H':{'0':'','1':'D'},
           'O':{'0':'','1':'17O','2':'18O'},
           'S':{'0':'','1':'33S','2':'34S','4':'36S'}}

massDict = {'C':{'0':12,'1':13.00335484},
            'N':{'0':14.003074,'1':15.00010889},
            'H':{'0':1.007825032,'1':2.014101778},
            'O':{'0':15.99491462,'1':16.99913175,'2':17.9991596},
            'S':{'0':31.97207117,'1':32.9714589,'2':33.96786701,'4':35.9670807}}

subDict = {'C':{'0':'','1':'13C'},
           'N':{'0':'','1':'15N'},
           'H':{'0':'','1':'D'},
           'O':{'0':'','1':'17O','2':'18O'},
           'S':{'0':'','1':'33S','2':'34S','4':'36S'}}

def computeMass(isotopologue, IDs):
    mass = 0
    for i in range(len(isotopologue)):
        if isotopologue[i] != 'x':
            element = IDs[i]
            mass += massDict[element][str(isotopologue[i])]
        
    return mass

def computeSubs(isotopologue, IDs):
    subs = []
    for i in range(len(isotopologue)):
        if isotopologue[i] != 'x':
            element = IDs[i]
            if subDict[element][str(isotopologue[i])] != '':
                subs.append(subDict[element][str(isotopologue[i])])
        
    return '/'.join(subs)

def UValueBySub(dictionary, sub):
    '''
    inefficient but user-friendly
    '''
    den = dictionary['000000000000000000000']['Conc']
    
    num = 0
    
    for i, v in dictionary.items():
        if v['Subs'] == sub:
            num += v['Conc']
            
    return num / den

def rpos(dictionary, pos):
    #pos is "position", a string corresponding to the integer position in the shorthand version of the molecule
    r = 0
    num = 0
    den = 0
    for i, value in dictionary.items():
        if i[pos] == '1':
            num += value
            
        #Key point here: ratio is 17O/16O, not 17O/Other O. Easy to get tripped up here. 
        if i[pos] == '0':
            den += value
    rN1 = num/den
    
    return rN1

def Usite(dictionary, pos):
    zeros = '00000000000000000000'
    num = list(['0']*20)
    num[pos] = '1'
    numerator = ''.join(num)
    
    U = dictionary[numerator]/dictionary[zeros]

    return U

In [3]:
subsDict = {'H':(0,1),'N':(0,1),'C':(0,1),'O':(0,1,2),'S':(0,1,2,4)}

##### INITIALIZE SITES #####
IDList = ['Cmethyl','Cgamma','Calphabeta','Ccarboxyl','Ocarboxyl1','Ocarboxyl2','Ssulfur','Namine','Hmethyl','Hgamma',
         'Halphabeta','Hamine','Hhydroxyl','Hprotonated']
elIDs = ['C','C','C','C','O','O','S','N','H','H','H','H','H','H']
numberAtSite = [1,1,2,1,1,1,1,1,3,2,3,2,1,1]

deltas = [-45,-35,-30,-25,-13,-13,2.5,10,-250,-100,0,100,250,0]

frag_full = [1,1,1,1,1,1,1,1,1,1,1,1,1,1]
frag_133 = [1,1,1,1,1,1,1,'x',1,1,1,'x',1,'x']
frag_104 = [1,1,1,'x','x','x',1,1,1,1,1,1,'x','x']
frag_102 = ['x',1,1,1,1,1,'x',1,'x',1,1,1,1,'x']
#88 and both 74 are incorrect but right stoich for predictions
#C3H8ON
#C2H4O2N
frag_88 = [1,1,1,'x','x','x',1,'x',1,1,'x',1,'x','x']
frag_74High = [1,'x',1,'x',1,'x','x',1,1,1,1,'x','x','x']
frag_74Low = [1,1,'x','x',1,1,'x',1,'x',1,'x',1,'x','x']
frag_61 = [1,1,'x','x','x','x',1,'x',1,1,'x','x','x','x']
frag_56 = ['x',1,1,'x','x','x','x',1,'x',1,1,'x',1,'x']

l = [elIDs, numberAtSite, deltas, frag_full, frag_133, frag_104, frag_102,
     frag_88, frag_74High, frag_74Low, frag_61, frag_56]

infoDf = pd.DataFrame(l, columns = IDList)
infoDf = infoDf.transpose()
infoDf.columns = ['IDS','Number','deltas','frag_full','frag_133','frag_104','frag 102',
                  'frag_88','frag_74High','frag_74Low','frag 61','frag 56']

In [4]:
infoDf

,IDS,Number,deltas,frag_full,frag_133,frag_104,frag 102,frag_88,frag_74High,frag_74Low,frag 61,frag 56
Cmethyl,C,1,-45,1,1,1,x,1,1,1,1,x
Cgamma,C,1,-35,1,1,1,1,1,x,1,1,1
Calphabeta,C,2,-30,1,1,1,1,1,1,x,x,1
Ccarboxyl,C,1,-25,1,1,x,1,x,x,x,x,x
Ocarboxyl1,O,1,-13,1,1,x,1,x,1,1,x,x
Ocarboxyl2,O,1,-13,1,1,x,1,x,x,1,x,x
Ssulfur,S,1,2.5,1,1,1,x,1,x,x,1,x
Namine,N,1,10,1,x,1,1,x,1,1,x,1
Hmethyl,H,3,-250,1,1,1,x,1,1,x,1,x
Hgamma,H,2,-100,1,1,1,1,1,1,1,1,1


In [5]:
siteList = [(x,y) for x,y in zip(elIDs, numberAtSite)]
siteElementsList = [site[0] * site[1] for site in siteList]
siteElements = ''.join(siteElementsList)

In [6]:
##### COMPUTE ALL ISOTOPOLOGUES (BIG A) #####
#Generate combinations of substitutions present at each site. Also compute the number of possibilities for 
#each substitution; for example, an H site with 2 atoms present can have the (0,1) substitution present 
#in two different ways. 
possibleSubs = []
numberSubs = []
for site in siteList:
    el = site[0]
    n = site[1]
    
    if n == 1:
        possibleSubs.append(subsDict[el])
        numberSubs.append([1] * len(subsDict[el]))
        
    else:
        subs = tuple(itertools.combinations_with_replacement(subsDict[el], n))
        possibleSubs.append(subs)
        
        #Count the number of occurences of each substitution. This code is inefficient but is only run on 
        #small sets, so ok for now. If we define a "site" with dozens of atoms in it, will need to revisit. 
        
        #To count the number of occurences, we hack the itertools.product implementation, below. As we make a 
        #list of tuples giving the possible substitutions at each site, we also make a list of lists giving 
        #the number of ocurrences of each tuple. For example, the first entry of "possibleSubs" is (0, 1). Each
        #can occur in 1 way, so the corresponding entry of "numberSubs" is [1,1]. Because the structure of 
        #"numberSubs" is exactly the same as "possibleSubs", when we run the optimized itertools.product function
        #on both, we end up with a list that maps each isotopologue to its number of occurrences. 
        siteIsotopicStructures = itertools.product(subsDict[el],repeat = n)
        sort = [sorted(list(x)) for x in siteIsotopicStructures]
        
        counts = []
        for siteComposition in subs:
            c = 0
            for isotopeStructure in sort:
                if list(siteComposition) == isotopeStructure:
                    c += 1
            counts.append(c)
            
        processedCounts = [[x] * n for x in counts]
            
        numberSubs.append(processedCounts)
        
#Compute all isotopologues (big A). For methionine: 663552, correct number based on hand computation.
#268 ms to compute

#if calculating numbers as well, 4.45 seconds. Compare to 1.57 seconds for calculating without multiatomic sites;
#we actually lose here by using multiatomic sites as we need to track more things. It is likely possible to
#optimize this. However, for this application, the major time cost is the calculation of individual isotopologue
#concentrations; here, having fewer isotopologues to track helps a lot. So the extra seconds spent here are worth
#it. 
#Optimizing this code could lead to big savings for large molecules.

i = 0
possibleIsotopologues = []
numberIsotopologues = []
for isotopologue in itertools.product(*possibleSubs):
    possibleIsotopologues.append(isotopologue)

for isotopologue in itertools.product(*numberSubs):
    flat = [x[0] if type(x) == list else x for x in isotopologue]
    n = np.array(flat).prod()
    
    numberIsotopologues.append(n)

In [7]:
##### COMPUTE ISOTOPOLOGUE CONCENTRATIONS ---Put possible concentrations in an Array#####
#Calculate the concentration of each isotopologue using input delta values
#Start dataFrame with to track and easily visualize input information


##Calculate the concentration of each isotopologue using input delta values
#determine concentration at each site
concentrationList = []
for index in range(len(elIDs)):
    element = cleanUpElID(elIDs[index])
    delta = deltas[index]
    concentration = op.deltaToConcentration(element, delta)
    concentrationList.append(concentration)
    
#put site-specific concentrations into a workable form
unsub = []
M1 = []
M2 = []
M3 = []
M4 = []
for concentration in concentrationList:
    unsub.append(concentration[0])
    M1.append(concentration[1])
    M2.append(concentration[2])
    M3.append(0)
    M4.append(concentration[3])

concentrationArray = np.array(unsub), np.array(M1), np.array(M2), np.array(M3), np.array(M4)

1.57 seconds per loop

In [8]:
##### COMPUTE ISOTOPOLOGUE CONCENTRATIONS ---Compute concentrations#####
d = {}
for i, isotopologue in enumerate(possibleIsotopologues):
    number = numberIsotopologues[i]
    isotopeConcList = []
    for index, value in enumerate(isotopologue):
        if type(value) == tuple:
            isotopeConc = [concentrationArray[sub][index] for sub in value]
            isotopeConcList += isotopeConc
        else:
            isotopeConc = concentrationArray[value][index]
            isotopeConcList.append(isotopeConc)      
        
    isotopologueConc = np.array(isotopeConcList).prod()
    
    string = ''.join(map(str, isotopologue))
    
    d[string] = {'Conc':isotopologueConc * number,'num':number}
    
    

In [9]:
bySub = {}
for i, v in d.items():
    condensed = condenseStr(i)
    Subs = ''.join([uEl(element, int(number)) for element, number in zip(siteElements, condensed)])
    if Subs not in bySub:
        bySub[Subs] = {'Number': 0, 'Full': [],'Conc': 0, 'Mass': [], 'Condensed': []}
    bySub[Subs]['Number'] += v['num']
    bySub[Subs]['Full'].append(i)
    bySub[Subs]['Conc'] += v['Conc']
    bySub[Subs]['Mass'].append(np.array(list(map(int,condensed))).sum())
    bySub[Subs]['Condensed'].append(condensed)
    

In [10]:
byCondensed = {}
siteElementsList = [site[0] * site[1] for site in siteList]
siteElements = ''.join(siteElementsList)
for i, v in d.items():
    condensed = condenseStr(i)
    byCondensed[condensed] = {}
    byCondensed[condensed]['Number'] = v['num']
    byCondensed[condensed]['full'] = i
    byCondensed[condensed]['Conc'] = v['Conc']
    byCondensed[condensed]['Mass'] = np.array(list(map(int,condensed))).sum()
    byCondensed[condensed]['Subs'] = ''.join([uEl(element, int(number)) for element, number in zip(siteElements, condensed)])
    
    

In [11]:
stochD = copy.deepcopy(byCondensed)
clumpD = copy.deepcopy(byCondensed)

In [12]:
COClump = 1.4637E-09
clumpD['000010100000000000000']['Conc'] += COClump
clumpD['000011000000000000000']['Conc'] += COClump
clumpD['000010000000000000000']['Conc'] -= COClump
clumpD['000000100000000000000']['Conc'] -= COClump
clumpD['000001000000000000000']['Conc'] -= COClump
clumpD['000000000000000000000']['Conc'] += COClump

In [13]:
NHClump = -2.2662E-10

clumpD['000000001000000000100']['Conc'] += NHClump * 2
clumpD['000000001000000000000']['Conc'] -= NHClump
clumpD['000000000000000000100']['Conc'] -= NHClump * 2
clumpD['000000000000000000000']['Conc'] += NHClump

In [14]:
SDClump = 0
clumpD['000000010000010000000']['Conc'] += SDClump * 2
clumpD['000000000000010000000']['Conc'] -= SDClump * 2
clumpD['000000010000000000000']['Conc'] -= SDClump
clumpD['000000000000000000000']['Conc'] += SDClump

In [15]:
a = stochD['000010100000000000000']['Conc'] / stochD['000000000000000000000']['Conc']
b = clumpD['000010100000000000000']['Conc'] / clumpD['000000000000000000000']['Conc']
capDelta = 1000 * (b/a -1)
print(capDelta)

a = stochD['000011000000000000000']['Conc'] / stochD['000000000000000000000']['Conc']
b = clumpD['000011000000000000000']['Conc'] / clumpD['000000000000000000000']['Conc']
capDelta = 1000 * (b/a -1)
print(capDelta)

0.40003609055050404
0.40003609055050404


In [16]:
a = stochD['000000001000000000100']['Conc'] / stochD['000000000000000000000']['Conc']
b = clumpD['000000001000000000100']['Conc'] / clumpD['000000000000000000000']['Conc']
capDelta = 1000 * (b/a -1)
print(capDelta)

-0.39999266285539736


In [17]:
a = stochD['000000010000010000000']['Conc'] / stochD['000000000000000000000']['Conc']
b = clumpD['000000010000010000000']['Conc'] / clumpD['000000000000000000000']['Conc']
capDelta = 1000 * (b/a -1)
print(capDelta)

-1.3889789318710655e-06


In [18]:
##Pull out M0 substitutions only
M0 = {}
for i, v in clumpD.items():
    if v['Mass'] == 0:
        M0[i] = v
    
##Pull out M0 substitutions only
M1 = {}
for i, v in clumpD.items():
    if v['Mass'] == 1:
        M1[i] = v
    
##Pull out M0 substitutions only
M2 = {}
for i, v in clumpD.items():
    if v['Mass'] == 2:
        M2[i] = v
    
##Pull out M0 substitutions only
M3 = {}
for i, v in clumpD.items():
    if v['Mass'] == 3:
        M3[i] = v
    
##Pull out M0 substitutions only
M4 = {}
for i, v in clumpD.items():
    if v['Mass'] == 4:
        M4[i] = v
    

In [19]:
###Initialize information to simulate measurement
allMeasurementInfo = {}

siteElementsList = [site[0] * site[1] for site in siteList]
siteElements = ''.join(siteElementsList)
massSelectedFragment = [M1, M2, M3, M4]
massSelected = [M1, M2, M3, M4]
massKeysFragment = ['M1','M2','M3','M4']
massKeys = ['M1','M2','M3','M4']

fragCondensedList = [frag_133, frag_104, frag_102, frag_61, frag_56]
fragments = [expandFrag(x, numberAtSite) for x in fragCondensedList]
fragKeys = ['133','104','102','61','56']

In [20]:
#Simulate M+N experiments

for i, massSelection in enumerate(massSelectedFragment):
    if massKeys[i] not in allMeasurementInfo:
        allMeasurementInfo[massKeysFragment[i]] = {}
    
    for j, fragment in enumerate(fragments):
        fragmentedDict = {}
        for isotopologue, value in massSelection.items():
            frag = [fragMult(x,y) for x, y in zip(fragment, isotopologue)]
            newIsotopologue = ''.join(frag)
            if newIsotopologue not in fragmentedDict:
                fragmentedDict[newIsotopologue] = 0
            fragmentedDict[newIsotopologue] += value['Conc']

        selectedIsotopologues = fragmentedDict
        predictSpectrum = {}

        for key, item in selectedIsotopologues.items():
            subs = computeSubs(key, siteElements)

            if subs not in predictSpectrum:
                predictSpectrum[subs] = {'Abs. Abundance':0}

            predictSpectrum[subs]['Abs. Abundance'] += item

        totalAbundance = 0
        for key, item in predictSpectrum.items():
            totalAbundance += item['Abs. Abundance']

        for key, item in predictSpectrum.items():
            item['Rel. Abundance'] = item['Abs. Abundance'] / totalAbundance

        shortSpectrum = {}
        for x, v in predictSpectrum.items():
            if v['Rel. Abundance'] > 0.001:
                shortSpectrum[x] = v
        
        allMeasurementInfo[massKeys[i]][fragKeys[j]] = shortSpectrum

In [21]:
#simulate mass selection without fragmentation
for i, massSelection in enumerate(massSelected):
    string = "Full"
    if string not in allMeasurementInfo:
        allMeasurementInfo[string] = {}
        
    massDict = {}    
    totalConc = 0
        
    for j, v in massSelection.items():
        sub = v['Subs']
        if sub not in massDict:
            massDict[sub] = {'Abs. Abundance':0}
            
        massDict[sub]['Abs. Abundance'] += v['Conc']
        totalConc += v['Conc']
        
    for j, v in massDict.items():
        massDict[j]['Rel. Abundance'] = massDict[j]['Abs. Abundance'] / totalConc
        if massDict[j]['Rel. Abundance'] > 0.0000001:
            allMeasurementInfo[string][j] = v

In [22]:
outputDict = {}
fullMolecule = ['13C','15N','33S','34S','17O','18O','D']
ratios = ['D/34S','17O/34S','33S/34S','15N/34S','13C13C/34S',
         '18O/34S','13C33S/34S','13C18O/34S','13C34S/34S',
         '17O34S/34S','34S15N/34S','34SD/13C34S','18O33S/13C34S',
         '13C13C34S/13C34S','18O34S/13C34S','36S/13C34S',
         '13C34S15N/36S','13C34SD/36S','18O18O/36S',
         '13C13C18O/36S']

for key in fullMolecule:
    v = UValueBySub(clumpD, key)
    
    outputDict[key] = v
    
for key, value in allMeasurementInfo.items():
    if key in massKeys:
        for fragment, data in value.items():
            totalAdjAbund = 0
            for sub, abundance in data.items():
                outputDict[key + " " + fragment + ' ' + sub] = {'Abs. Abundance':abundance['Abs. Abundance'],
                                                    'Rel. Abundance':abundance['Rel. Abundance'],
                                                    'Adj. Rel. Abundance':0}
                totalAdjAbund += abundance['Abs. Abundance']
                
            for sub, abundance in data.items():
                outputDict[key + " " + fragment + ' ' + sub]['Adj. Rel. Abundance'] = abundance['Abs. Abundance'] / totalAdjAbund
                
for key, value in allMeasurementInfo.items():
    if key not in massKeys:
        for uValue in ratios:
            num, den = uValue.split('/')
            
            fraction = value[num]['Abs. Abundance']/value[den]['Abs. Abundance']
            
            outputDict[uValue] = fraction
    


In [23]:
out = pd.DataFrame.from_dict(outputDict).T

In [24]:
out

,Abs. Abundance,Rel. Abundance,Adj. Rel. Abundance
13C,0.054332,0.054332,0.054332
15N,0.003713,0.003713,0.003713
33S,0.007915,0.007915,0.007915
34S,0.044959,0.044959,0.044959
17O,0.000750,0.000750,0.000750
...,...,...,...
36S/13C34S,0.043495,0.043495,0.043495
13C34S15N/36S,0.085361,0.085361,0.085361
13C34SD/36S,0.041183,0.041183,0.041183
18O18O/36S,0.035976,0.035976,0.035976


In [25]:
out.to_csv('SyntheticMeasurements.csv')

In [26]:
#####PREDICT FULL SPECTRUM (NO FRAGMENT) #####
#####CUT OFF ISOTOPOLOGUES WITH LOW REL ABUNDANCE #####
selectedIsotopologues = M3
lowAbundanceCutOff = 0.01
massError = -0.000


predictSpectrum = {}
for key, item in selectedIsotopologues.items():
    mass = computeMass(key, siteElements)
    correctedMass = mass + massError
    subs = computeSubs(key, siteElements)
    
    if correctedMass not in predictSpectrum:
        predictSpectrum[correctedMass] = {'Abs. Abundance':0}
        
        if 'Sub' not in predictSpectrum[correctedMass]:
            predictSpectrum[correctedMass]['Sub'] = subs
            
    predictSpectrum[correctedMass]['Abs. Abundance'] += item['Conc']
    
totalAbundance = 0
for key, item in predictSpectrum.items():
    totalAbundance += item['Abs. Abundance']
    
massPlot = []
relAbundPlot = []
subPlot = []
for key, item in predictSpectrum.items():
    item['Rel. Abundance'] = item['Abs. Abundance'] / totalAbundance
    massPlot.append(key)
    relAbundPlot.append(item['Rel. Abundance'])
    subPlot.append(item['Sub'])
    
fig, ax = plt.subplots(figsize = (20,4))
massPlotcutOff = []
subPlotcutOff = []
for i in range(len(massPlot)):
    if relAbundPlot[i] > lowAbundanceCutOff:
        ax.vlines(massPlot[i], 0, relAbundPlot[i])
        massPlotcutOff.append(massPlot[i])
        subPlotcutOff.append(subPlot[i])
ax.set_xticks(massPlotcutOff)
labels = [str(round(x,5)) +'\n' + y for x,y in zip(massPlotcutOff,subPlotcutOff)]
ax.set_xticklabels(labels,rotation = 45);
    

KeyError: 'C'

In [ ]:
##### PREDICT SPECTRUM WITH FRAGMENTATION #####
selectedIsotopologues = M3
lowAbundanceCutOff = 0.001
massError = 0.000
fragment = expandFrag(frag_104, numberAtSite)
bounds = None



##### Fragment Isotopologues #####
fragmentedDict = {}
for isotopologue, data in selectedIsotopologues.items():
    frag = [fragMult(x,y) for x, y in zip(fragment, isotopologue)]
    newIsotopologue = ''.join(frag)
    if newIsotopologue not in fragmentedDict:
        fragmentedDict[newIsotopologue] = 0
    fragmentedDict[newIsotopologue] += data['Conc']

selectedIsotopologues = fragmentedDict


##### Predict Spectrum #####
predictSpectrum = {}
for key, item in selectedIsotopologues.items():
    mass = computeMass(key, siteElements)
    correctedMass = mass + massError
    subs = computeSubs(key, siteElements)
    
    if correctedMass not in predictSpectrum:
        predictSpectrum[correctedMass] = {'Abs. Abundance':0}
        
        if 'Sub' not in predictSpectrum[correctedMass]:
            predictSpectrum[correctedMass]['Sub'] = subs
            
    predictSpectrum[correctedMass]['Abs. Abundance'] += item
    
totalAbundance = 0
for key, item in predictSpectrum.items():
    totalAbundance += item['Abs. Abundance']
    
massPlot = []
relAbundPlot = []
subPlot = []
for key, item in predictSpectrum.items():
    item['Rel. Abundance'] = item['Abs. Abundance'] / totalAbundance
    massPlot.append(key)
    relAbundPlot.append(item['Rel. Abundance'])
    subPlot.append(item['Sub'])
    
fig, ax = plt.subplots(figsize = (10,4))
massPlotcutOff = []
subPlotcutOff = []
for i in range(len(massPlot)):
    if relAbundPlot[i] > lowAbundanceCutOff:
        ax.vlines(massPlot[i], 0, relAbundPlot[i])
        massPlotcutOff.append(massPlot[i])
        subPlotcutOff.append(subPlot[i])
ax.set_xticks(massPlotcutOff)
labels = [str(round(x,5)) +'\n' + y for x,y in zip(massPlotcutOff,subPlotcutOff)]
ax.set_xticklabels(labels,rotation = 45);
if bounds != None:
    ax.set_xlim(bounds[0],bounds[1])
ax.set_xlim(107.035, 107.07)
    

In [ ]:
###See below for an interesting sidenote...

In [ ]:
f61 = {}
frag_61Full = expandFrag(frag_61, numberAtSite)
for isotopologue, v in clumpD.items():
    frag = [fragMult(x,y) for x, y in zip(frag_61Full, isotopologue)]
    newIsotopologue = ''.join(frag)
    if newIsotopologue not in f61:
        f61[newIsotopologue] = {'Conc':0,'Number':0,'Sources':0,'Mass':0,'Subs':''}
    f61[newIsotopologue]['Conc'] += v['Conc']
    f61[newIsotopologue]['Number'] += v['Number']
    f61[newIsotopologue]['Sources'] += 1
    f61[newIsotopologue]['Mass'] = computeMass(newIsotopologue, siteElements)
    f61[newIsotopologue]['Subs'] = computeSubs(newIsotopologue, siteElements)

In [ ]:
f61

In [ ]:
subList = []
numList = []
sourceList = []
iD = []
for i, v in f61.items():
    subList.append(v['Subs'])
    numList.append(v['Number'])
    sourceList.append(v['Sources'])
    iD.append(i)
    
    

In [ ]:
###The number of isotopologues contributing to every fragment of the 61 fragment of methionine. 
###Each is a scalar multiple of some constant; the scalar depends on the symmetry number of the isotopologue. 
###I don't know what to do with this information, but it is interesting. At some point there may be a clever
###computational way to take advantage of it. 

###Something similar occurs with the fine isotope structure of directly fragmented peaks. 

plt.scatter(range(len(numList)),numList)

In [ ]:
###Here, divide each by the number of sources; i.e. the number of sources is equal to the scalar. 
plt.scatter(range(len(sourceList)),sourceList)

In [ ]:
3456 * len(sourceList)

In [ ]:
len(sourceList)